In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn import metrics
import math
import xgboost as xgb
import scipy as sp
from scipy import stats

/home/xcheng/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#original simulation data generation
#input parameters: sample size, weight in the equation, max value of X (value range), tag to indicate the function between X and Y (0:Linear;1:Power;2:Trigonometric)
#output results: original data of generated X, original data of the corresponding generated Y
def generation(N,W,X_max,L):
    X_data=[]
    Y_data=[]
    
    #Linear, y=wx+b
    if(L==0):
        for i in range(N):
            x=random.random()*X_max
            b=random.random()*100
            y=W*x+b
        
            X_data.append((x,L))
            Y_data.append(y)
    
    #Power,y=wx^2+b
    elif(L==1):
        for i in range(N):
            x=random.random()*X_max
            b=random.random()*100
            y=W*x*x+b
        
            X_data.append((x,L))
            Y_data.append(y)

    #Trigonometric,y=w(sin(x)+1)+b
    elif(L==2):
        for i in range(N):
            x=random.random()*X_max
            b=random.random()*100
            y=W*(math.sin(x)+1)+b
        
            X_data.append((x,L))
            Y_data.append(y)
     
    return X_data,Y_data  

In [3]:
#resampling data according to the distribution density
#input parameters: original data of X, original data of Y, the number of samples after resampling, intervals of X to present the distribution density, distribution density of Y
#output results: resampled data of X, the corresponding resampled data of Y
def resampling(data_x,data_y,N,density_x,density_y):
    result_x=[]
    result_y=[]
    
    #how many intervals
    length=len(density_x)
    
    #length of the interval
    x_start=density_x[0]
    x_range=1.0*(density_x[1]-density_x[0])
    
    datapool=[]#samples Y in each interval according to their values
    poolsize=[]#the number of samples in each interval
    for i in range(length):
        tmp=[]
        if(i==length-1):
            samplesize=N-np.sum(poolsize)
        else:
            samplesize=int(density_y[i]*N)
        poolsize.append(samplesize)
        datapool.append(tmp)
        
    for j in range(len(data_y)):
        num=int((data_y[j]-x_start)/x_range)
        
        if(num>=length):
            num=length-1
        datapool[num].append(j)
    
    idpool=[]#ids of resampled samples in each interval
    for i in range(length):
        tmpsize=poolsize[i]
        tmpid=np.random.choice(datapool[i],tmpsize,replace=True)
        for j in tmpid:
            idpool.append(j)
            
    #get the samples according to their ids
    random.shuffle(idpool)
    for i in idpool:
        result_x.append(data_x[i])
        result_y.append(data_y[i])
    
    return result_x,result_y

#generate the distribution density and sample the data according to the distribution density
#input parameters: the number of samples, original data of X, original data of Y, tag to indicate the distribution type (0: Normal distribution;1: Pareto distribution)
#output results: resampled data of X, the corresponding resampled data of Y
def densitysampling(N,input_x,input_y,D):
    
    #input_x_d=np.arange(start=np.min(input_y),stop=np.max(input_y),step=50)
    input_x_d=np.arange(start=np.min(input_y),stop=np.max(input_y),step=(np.max(input_y)-np.min(input_y))/20.0)
    
    #Normal distribution
    if(D==0):
        #input_y_d=stats.norm.pdf(x=input_x_d,loc=(np.max(input_y)-np.min(input_y))/2.0+np.min(input_y),scale=200)
        input_y_d=stats.norm.pdf(x=input_x_d,loc=(np.max(input_y)-np.min(input_y))/2.0+np.min(input_y),scale=(np.max(input_y)-np.min(input_y))/5.0)
    #Pareto distribution
    elif(D==1):
        input_y_d=stats.pareto.pdf(x=input_x_d,loc=np.min(input_y),scale=1,b=0.05)    
    
    #Normalization
    input_y_d=input_y_d/np.sum(input_y_d)

    new_x,new_y=resampling(input_x,input_y,N,input_x_d,input_y_d)
    
    return new_x,new_y

In [4]:
#evaluate the RMSE of forecasting on three data (entire data, data generated based on the first equation, data generated based on the second equation)
def getresult(real_1,pred_1,real_2,pred_2):
    totalreal=np.concatenate((np.array(real_1),np.array(real_2)),axis=0)
    totalpred=np.concatenate((np.array(pred_1),np.array(pred_2)),axis=0)
    
    mse=metrics.mean_squared_error(totalreal,totalpred)
    mse_1=metrics.mean_squared_error(real_1,pred_1)
    mse_2=metrics.mean_squared_error(real_2,pred_2)
    
    return math.sqrt(mse),math.sqrt(mse_1),math.sqrt(mse_2)
    
#train XGBoost model and get the evaluation results
#input parameter: the first eight parameters are the corresponding data, the last parameter indicates whether the predicted Y needs to be transformed back from its log transformation (1:transform;0:not) 
#output results: RMSE on entire data by global training, RMSE on data_1 by global training, RMSE on data_2 by global training, RMSE on entire data by local training,...
def modeltraining(x_train_1,x_train_2,y_train_1,y_train_2,x_test_1,x_test_2,y_test_1,y_test_2,logtag):
    totaltrain_x=np.concatenate((np.array(x_train_1),np.array(x_train_2)),axis=0)
    totaltest_x=np.concatenate((np.array(x_test_1),np.array(x_test_2)),axis=0)
    totaltrain_y=np.concatenate((np.array(y_train_1),np.array(y_train_2)),axis=0)
    totaltest_y=np.concatenate((np.array(y_test_1),np.array(y_test_2)),axis=0)
    
    #global training
    model_g=xgb.XGBRegressor(eval_metric='rmse')
    model_g.fit(totaltrain_x,totaltrain_y)
    g_pred_1=model_g.predict(x_test_1)
    g_pred_2=model_g.predict(x_test_2)
    
    #transform back from log transformation
    if(logtag==1):
        g_pred_1=np.power(np.e,g_pred_1/200.0)
        g_pred_2=np.power(np.e,g_pred_2/200.0)
#         for i in range(len(g_pred_1)):
#             g_pred_1[i]=math.pow(math.e,g_pred_1[i])
#         for j in range(len(g_pred_2)):
#             g_pred_2[j]=math.pow(math.e,g_pred_2[j])
    
    #local training
    model_l1=xgb.XGBRegressor(eval_metric='rmse')
    model_l1.fit(x_train_1,y_train_1)
    l_pred_1=model_l1.predict(x_test_1)
    
    model_l2=xgb.XGBRegressor(eval_metric='rmse')
    model_l2.fit(x_train_2,y_train_2)
    l_pred_2=model_l2.predict(x_test_2)
    
    #transform back from log transformation
    if(logtag==1):
        l_pred_1=np.power(np.e,l_pred_1/200.0)
        l_pred_2=np.power(np.e,l_pred_2/200.0)
#         for i in range(len(l_pred_1)):
#             l_pred_1[i]=math.pow(math.e,l_pred_1[i])
#         for j in range(len(l_pred_2)):
#             l_pred_2[j]=math.pow(math.e,l_pred_2[j])
    
    
    g_rmse,g_rmse_1,g_rmse_2=getresult(y_test_1,g_pred_1,y_test_2,g_pred_2)
    l_rmse,l_rmse_1,l_rmse_2=getresult(y_test_1,l_pred_1,y_test_2,l_pred_2)
    
    return g_rmse,g_rmse_1,g_rmse_2,l_rmse,l_rmse_1,l_rmse_2

In [5]:
#visulization the value distribution of data
def distribution(input_y):
    #length=int(np.max(input_y)/40.0)
    length=25
    Y_max=np.max(input_y)
    plt.hist(input_y,bins=length,density=True)
    plt.xlabel('Y value',fontsize=13)
    plt.ylabel('Density',fontsize=13)
    plt.show()

In [6]:
#main program

#sample size after sampling
N1=1000
N2=2000-N1

#sample size before sampling
N_raw=10000

#w for each equation
W1=5
W2=10
W3=500

#value range of X
X1_max=200
X2_max=10
X3_max=2*math.pi


#consider different sample sizes, different value ranges

#Nlist=[1800,1600,1400,1200,1000,800,600,400,200]

w1list=[5,10,20,50,100,200,500,1000]
# w2list=[10,20,40,100,200,400,1000,2000]
# w3list=[500,1000,2000,5000,10000,20000,50000,100000]

#for different sample size:
# for tt in Nlist:
#     N1=tt
#     N2=2000-N1
#     ...

for tt in w1list:
    
    g_total_rmse=[]
    g_rmse_1=[]
    g_rmse_2=[]
    l_total_rmse=[]
    l_rmse_1=[]
    l_rmse_2=[]

    times=100
    

    #Linear 0, Power 1, Trigonometric 2
    #for different value range, tt
    X_rawtrain_1,Y_rawtrain_1=generation(N_raw,tt,X1_max,0)
    X_rawtrain_2,Y_rawtrain_2=generation(N_raw,W2,X2_max,1)

    X_rawtest_1,Y_rawtest_1=generation(N_raw,tt,X1_max,0)
    X_rawtest_2,Y_rawtest_2=generation(N_raw,W2,X2_max,1)
    
    
    
    for i in range(times):
        #Normal 0, Pareto 1
        X_train_1,Y_train_1=densitysampling(N1,X_rawtrain_1,Y_rawtrain_1,0)
        X_train_2,Y_train_2=densitysampling(N2,X_rawtrain_2,Y_rawtrain_2,0)
        
        X_test_1,Y_test_1=densitysampling(N1,X_rawtest_1,Y_rawtest_1,0)
        X_test_2,Y_test_2=densitysampling(N2,X_rawtest_2,Y_rawtest_2,0)
        
        #to avoid the value is to small
        Y_train_1=np.log(Y_train_1)*200
        Y_train_2=np.log(Y_train_2)*200
#         Y_test_1=np.log(Y_test_1)
#         Y_test_2=np.log(Y_test_2)
        
        #tag=0 without log transformation; tag=1 with log transformation
        tag=1
    
        g,g1,g2,l,l1,l2=modeltraining(X_train_1,X_train_2,Y_train_1,Y_train_2,X_test_1,X_test_2,Y_test_1,Y_test_2,tag)
    
        g_total_rmse.append(g)
        g_rmse_1.append(g1)
        g_rmse_2.append(g2)
        l_total_rmse.append(l)
        l_rmse_1.append(l1)
        l_rmse_2.append(l2)
        
        #to show the process
        if(i%50==0):
            print(tt,i)
    #print the results (mean,std) of global training on entire data, data1, and data2; and results of local training on entire data, data1, and data2
    print(np.mean(g_total_rmse),np.std(g_total_rmse),';',np.mean(g_rmse_1),np.std(g_rmse_1),';',np.mean(g_rmse_2),np.std(g_rmse_2))
    print(np.mean(l_total_rmse),np.std(l_total_rmse),';',np.mean(l_rmse_1),np.std(l_rmse_1),';',np.mean(l_rmse_2),np.std(l_rmse_2))


5 0
5 50
32.35827041259325 0.7301295149769782 ; 31.979342719519327 0.6449793215786973 ; 32.719935933931204 1.2218591522740005
33.221431118183986 0.4816457628107591 ; 33.13960678427112 0.6457423494055771 ; 33.29568233410694 0.7334529776800378
10 0
10 50
32.94118707892555 1.5316070579057302 ; 33.14069579499674 2.6608864637882585 ; 32.6954238329041 0.7473142138863892
33.621241181773605 0.4717092315491878 ; 34.05670069550065 0.7041413117474029 ; 33.1733123957007 0.6303997554426797
20 0
20 50
34.43975943570456 1.9234005469853819 ; 35.911089743022174 3.3838368616845615 ; 32.831005798554266 0.8132275967391087
33.82633341967093 0.5229009661675429 ; 34.46345400840489 0.6675708101615035 ; 33.17021871916709 0.741480427880604
50 0
50 50
39.2982462304734 1.4421759291638105 ; 44.74386193007782 2.465524532417353 ; 32.922467769503776 0.9397769582828994
35.387610914552944 0.5735525761199142 ; 37.395582661302555 0.8839153315767405 ; 33.249769777272796 0.6825235976637521
100 0
100 50
53.10935451853888 2.